# Описание Домашнего Задания

Технический анализ

<b>Цель:</b>

В данном домашнем задании вы потренируетесь в построении торговой стратегии на базе технического анализа. Также вы построите первую модель оценки эффективности торговой стратегии.


<b>Описание/Пошаговая инструкция выполнения домашнего задания:</b>

Уважаемый студент!

…Итак, данные собраны и можно начинать анализ. Коллеги посоветовали вам выделить паттерны и на их основе построить торговую стратегию.


Вы решили посмотреть сначала самые простые паттерны на основе скользящих средних и затем дополнить их более сложными шаблонами, такими как паттерны разворота, продолжения и свечного анализа.


Также вы понимаете, что нужен инструмент оценки эффективности и поскольку, у вас уже есть данные прошлых периодов, вы решаете создать инструмент оценки эффективности новой стратегии, пока у вас есть немного свободного времени.

На основании вышесказанного, вам необходимо построить модель, которая будет принимать решение о торговых операциях по одной или группе бумаг и криптовалют, и оценить ее эффективность на отложенных данных.


<b>Вам предлагается на основе представленной информации:</b>

1) Создать код на Python, который разделит ваши данные на тренировочный, тестовый и валидационный наборы данных.
2) Построить одну или несколько моделей на основе паттернов технического анализа, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам.
3) Провести подбор гиперпараметров моделей с использованием обучающей и тестовой выборок.
4) Провести финальное тестирование построенных торговых стратегий на валидационном наборе данных и сравнить их между собой.
5) Сформировать дашборд, показывающий эффективность различных стратегий во времени.

# 1. Код на Python, разделяющий данные на тренировочный, тестовый и валидационный наборы данных

Принимаемые допущения:
- данные будем использовать из кэша (загрузка - отдельной, уже реализованной функцией);
- в данных должен быть ровно один тикер (поскольку рассматриваемые в курсе решения не добрались до управления портфелем, + вероятно это впоследствии возможно будет распараллелить);
- функционал разрабатываем на примере данных Ethereum ("ETH-USD"), но делаем тикер управляемым параметром;
- границы train/test/valid датасетов должны быть управляемыми, для дальнейшего тестирования на различных временных периодах;
- в X_test / X_valid не должно быть "протекающих" признаков (Open / High / Low / Volume), при этом оставим возможность того, чтобы использовались другие признаки;
- вероятно, для сравнимости результатов тестирования длины тестовой и валидационной выборок должны быть одинаковыми, вне зависимости от длины обучающей (разные тикеры / разные периоды тестирования), поэтому реализация будет через определение временных границ, а не train_size / test_size из sklearn.model_selection_train_test_split;

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

from typing import Optional

import pandas as pd

from src.core import utils

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()
# SUBSTITUTE FOR TESTING
# TODO

In [2]:
# Params for data download
TICKER = "ETH-USD"
START_DT = '2010-01-01' # get as long as possible
END_DT = config.END_DT # defaults to today
INTERVAL = '1d'

# params for split function
train_start = None
train_end = "2023-01-01"
test_end = "2024-01-01"
valid_end = "2025-01-01" #config.END_DT

In [3]:
# Обновим данные в кэширующей базе
utils.update_tickers_data(
    tickers=TICKER, start_dt=START_DT, end_dt=END_DT, interval=INTERVAL
)

[INFO   ] 2025-03-22@20:00:23: Made sure table in database for interval='1d' exists
[INFO   ] 2025-03-22@20:00:23: Checking already available data...
[INFO   ] 2025-03-22@20:00:26: 0 tickers have no data at all
[INFO   ] 2025-03-22@20:00:26: 1 tickers lack history in start part
[INFO   ] 2025-03-22@20:00:26: 1 tickers lack history in end part
[INFO   ] 2025-03-22@20:00:26: Updating tickers lacking early history data...
[INFO   ] 2025-03-22@20:00:26: Downloading data using yfinance


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[INFO   ] 2025-03-22@20:00:26: Downloaded data shape: (0, 5)
[INFO   ] 2025-03-22@20:00:26: reshaped: (0, 7)
[INFO   ] 2025-03-22@20:00:27: Updating tickers lacking late history data...
[INFO   ] 2025-03-22@20:00:27: Downloading data using yfinance
[*********************100%***********************]  1 of 1 completed
[INFO   ] 2025-03-22@20:00:27: Downloaded data shape: (1, 5)
[INFO   ] 2025-03-22@20:00:27: reshaped: (1, 7)
[INFO   ] 2025-03-22@20:00:28: Data in caching DB updated


In [4]:
# Загрузим данные для дальнейшего использования
ticker_data = utils.get_history(
    tickers=TICKER,
    start=START_DT,
    end=END_DT,
    interval=INTERVAL,
    update_cache=False,
)

ticker_data.sample(3)

[INFO   ] 2025-03-22@20:00:29: Getting history from local cache DB...
[INFO   ] 2025-03-22@20:00:30: Got history of shape (2690, 7), 0 NaNs


,Date,Ticker,Open,Low,High,Close,Volume
2551,2024-03-08 00:00:00.000000,ETH-USD,3874.830811,3828.363281,3998.826416,3892.061035,2.613549e+10
1980,2022-08-15 00:00:00.000000,ETH-USD,1936.760498,1881.856812,2007.210327,1904.228149,2.034993e+10
2277,2023-06-08 00:00:00.000000,ETH-USD,1832.513550,1830.165039,1861.136108,1846.301880,4.536042e+09


In [5]:
def train_test_valid_split(
    ticker_data: pd.DataFrame,
    train_start: Optional[str],
    train_end: str,
    test_end: str,
    valid_end: str,
    drop_leaky:bool=True
) -> tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame
]:
    """
    Split ticker data to training, testing and validation datasets
    """
    logger.info("Splitting ticker data to train/test/validation parts")
    # 0. Make sure that Date is ascending. Also reset index
    ticker_data["Date"] = pd.to_datetime(ticker_data["Date"])
    ticker_data = ticker_data.sort_values(by=["Date"], ascending=True).reset_index(
        drop=True
    )

    # 1. Drop leaky columns
    try:
        ticker_data.drop(columns=["Ticker"], inplace=True)
    except:
        pass
    if drop_leaky:
        for col in ["Open", "Low", "High", "Volume"]:
            try:
                ticker_data.drop(columns=col, inplace=True)
            except:
                pass

    # 2. Perform train/test/valid split based on 'Date'
    # we don't need anything after validation end
    ticker_data = ticker_data[ticker_data["Date"] < valid_end].reset_index(drop=True)
    # in case train_start is defined - cut it
    if train_start is not None:
        ticker_data = ticker_data[ticker_data["Date"] >= train_start].reset_index(
            drop=True
        )
    # Train parts
    X_train = (
        ticker_data[ticker_data["Date"] < train_end]
        .drop(columns=["Close"])
        .reset_index(drop=True)
    )
    y_train = ticker_data[ticker_data["Date"] < train_end]["Close"].reset_index(
        drop=True
    )
    # Test parts
    X_test = (
        ticker_data[
            (ticker_data["Date"] >= train_end) & (ticker_data["Date"] < test_end)
        ]
        .drop(columns=["Close"])
        .reset_index(drop=True)
    )
    y_test = ticker_data[
        (ticker_data["Date"] >= train_end) & (ticker_data["Date"] < test_end)
    ]["Close"].reset_index(drop=True)
    # Validation parts
    X_val = (
        ticker_data[ticker_data["Date"] >= test_end]
        .drop(columns=["Close"])
        .reset_index(drop=True)
    )
    y_val = ticker_data[ticker_data["Date"] >= test_end]["Close"].reset_index(drop=True)

    return X_train, y_train, X_test, y_test, X_val, y_val

In [6]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_valid_split(
    ticker_data, train_start, train_end, test_end, valid_end
)

[INFO   ] 2025-03-22@20:00:35: Splitting ticker data to train/test/validation parts


# 2. Модель на основе паттернов технического анализа, которая будет принимать торговые решения по бумагам SnP500 и/или криптовалютам

**Учитывая что модель должна работать по паттернам техического анализа - неочевидно зачем в предыдущем пункте вообще нужна обучающая выборка, ведь стратегия тестируется сразу на тестовой выборке**

План:
- реализовать несколько классов, наследуясь от backtesting.Strategy;
- гиперпараметры в них - это параметры для расчета конкретных показателей (например, длина окна скользящей средней);
- реализованные модели сохранить в /app/src/strategy в отдельные модули для дальнейшего использования;

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

import pandas as pd

import talib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from src.core import utils

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()
# SUBSTITUTE FOR TESTING

In [2]:
# Params for data download
TICKER = "ETH-USD"
START_DT = "2010-01-01"  # get as long as possible
END_DT = config.END_DT  # defaults to today
INTERVAL = "1d"

# params for split function
train_start = None
train_end = "2023-01-01"
test_end = "2024-01-01"
valid_end = "2025-01-01"  # config.END_DT

In [3]:
# Возьмём данные - с разбивкой на train/test/valid
ticker_data = utils.get_history(
    tickers=TICKER,
    start=START_DT,
    end=END_DT,
    interval=INTERVAL,
    update_cache=False,
)
X_train, y_train, X_test, y_test, X_val, y_val = utils.train_test_valid_split(
    ticker_data, train_start, train_end, test_end, valid_end, drop_leaky=False
)

[INFO   ] 2025-03-22@20:47:15: Getting history from local cache DB...
[INFO   ] 2025-03-22@20:47:15: Got history of shape (2690, 7), 0 NaNs
[INFO   ] 2025-03-22@20:47:15: Splitting ticker data to train/test/validation parts


In [4]:
# Для трансформации данных к виду, ожидаемому библоиотекой Backtesting, используем сервисную функцию
def transform_for_backtesting(y_test: pd.DataFrame, X_test:pd.DataFrame) -> pd.DataFrame:
    """
    Transform data after train/test split to the format fit for Backtesing library
    """
    bt_df = pd.concat([X_test, y_test], axis=1)
    # bt_df["Open"] = 0
    # bt_df["High"] = 0
    # bt_df["Low"] = 0
    bt_df["Date"] = pd.to_datetime(bt_df["Date"])
    bt_df.set_index("Date", inplace=True)
    return bt_df

In [5]:
# Стратегия №1: пересечение скользящих средних (быстрой и медленной)
class SmaCross(Strategy):
    """
    Basis Simple Moving Average crossover strategy
    """
    ma_fast_periods = 10
    ma_slow_periods = 20

    def init(self)->None:
        price = self.data['Close']
        self.ma1 = self.I(talib.SMA, price, self.ma_fast_periods)
        self.ma2 = self.I(talib.SMA, price, self.ma_slow_periods)

    def next(self)->None:
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma2):
            self.sell()

In [6]:
# Test a strategy (SmaCross)
bt = Backtest(
    transform_for_backtesting(y_test, X_test),
    SmaCross,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)
stats = bt.run(ma_fast_periods=3, ma_slow_periods=14)
stats[:23]

Start                     2023-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   89.315068
Equity Final [$]                133197.215895
Equity Peak [$]                 138838.806227
Commissions [$]                   7892.035338
Return [%]                          33.197216
Buy & Hold Return [%]               47.124591
Return (Ann.) [%]                   33.197216
Volatility (Ann.) [%]               61.035247
CAGR [%]                            33.302154
Sharpe Ratio                         0.543902
Sortino Ratio                         1.19076
Calmar Ratio                         1.114676
Alpha [%]                           -8.268368
Beta                                 0.879914
Max. Drawdown [%]                  -29.781949
Avg. Drawdown [%]                     -6.2029
Max. Drawdown Duration      232 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                          

In [7]:
bt.plot(superimpose=False)

INFO:bokeh.io.state:Session output file 'SmaCross_ma_fast_periods-3,ma_slow_periods-14_.html' already exists, will be overwritten.
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1317', ...)

In [8]:
# Стратегия №2: MACD
class MACDStrategy(Strategy):
    """
    Moving Average Convergence-Divergence strategy
    """

    fastperiod = 12
    slowperiod = 26
    signalperiod = 9

    def init(self) -> None:
        price = self.data["Close"]

        # Revealing object
        self.signal = self.I(
            talib.MACD,
            price,
            fastperiod=self.fastperiod,
            slowperiod=self.slowperiod,
            signalperiod=self.signalperiod,
        )

    def next(self) -> None:
        if self.signal[0][-1] > self.signal[1][-1]:  # self.signal[-1] == 1:
            if not self.position.is_long:
                self.buy()
        elif self.signal[0][-1] < self.signal[1][-1]:  # self.signal[-1] == -1:
            if not self.position.is_short:
                self.sell()

In [9]:
# Test a strategy (MACDStrategy)
bt = Backtest(
    transform_for_backtesting(y_test, X_test),
    MACDStrategy,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)
stats = bt.run(fastperiod=12, slowperiod=26, signalperiod=9)
stats[:23]

Start                     2023-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   90.136986
Equity Final [$]                 79124.413613
Equity Peak [$]                 112136.229905
Commissions [$]                   9012.901032
Return [%]                         -20.875586
Buy & Hold Return [%]               37.046236
Return (Ann.) [%]                  -20.875586
Volatility (Ann.) [%]               35.387794
CAGR [%]                           -20.926468
Sharpe Ratio                        -0.589909
Sortino Ratio                       -0.696776
Calmar Ratio                        -0.701459
Alpha [%]                          -21.595475
Beta                                 0.019432
Max. Drawdown [%]                  -29.760248
Avg. Drawdown [%]                   -11.59813
Max. Drawdown Duration      181 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                          

In [10]:
bt.plot(superimpose=False)

INFO:bokeh.io.state:Session output file 'MACDStrategy_fastperiod-12,slowperiod-26,signalperiod-9_.html' already exists, will be overwritten.


c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1701', ...)

In [11]:
# Стратегия №3: STOCH
class STOCHStrategy(Strategy):
    """
    Stochastic Oscillator-based strategy
    """

    fastk_period = 14
    slowk_period = 7
    slowk_matype = 0
    slowd_period = 7
    slowd_matype = 0

    def init(self) -> None:
        high = self.data['High']
        low = self.data['Low']
        close = self.data['Close']

        # Revealing object
        self.signal = self.I(
            talib.STOCH,
            high,
            low,
            close,
            fastk_period=self.fastk_period,
            slowk_period=self.slowk_period,
            slowk_matype=self.slowk_matype,
            slowd_period=self.slowd_period,
            slowd_matype=self.slowd_matype,
        )

    def next(self) -> None:
        if self.signal[0][-1] > self.signal[1][-1]:  # self.signal[-1] == 1:
            if not self.position.is_long:
                self.buy()
        elif self.signal[0][-1] < self.signal[1][-1]:  # self.signal[-1] == -1:
            if not self.position.is_short:
                self.sell()

In [12]:
# Test a strategy (STOCHStrategy)
bt = Backtest(
    transform_for_backtesting(y_test, X_test),
    STOCHStrategy,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)
stats = bt.run(fastk_period=14, slowk_period = 7, slowk_matype = 0, slowd_period = 7, slowd_matype = 0)
stats[:23]

Start                     2023-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   90.684932
Equity Final [$]                 81295.286939
Equity Peak [$]                 101711.359798
Commissions [$]                  12366.313239
Return [%]                         -18.704713
Buy & Hold Return [%]               42.315683
Return (Ann.) [%]                  -18.704713
Volatility (Ann.) [%]               37.435525
CAGR [%]                           -18.750949
Sharpe Ratio                        -0.499651
Sortino Ratio                       -0.631175
Calmar Ratio                        -0.517654
Alpha [%]                          -22.056002
Beta                                 0.079197
Max. Drawdown [%]                  -36.133639
Avg. Drawdown [%]                  -20.320724
Max. Drawdown Duration      335 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                          

In [13]:
bt.plot(superimpose=False)

INFO:bokeh.io.state:Session output file 'STOCHStrategy_fastk_period-14,slowk_period-7,slowk_matype-0,slowd_period-7,slowd_matype-0_.html' already exists, will be overwritten.


c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2079', ...)

In [14]:
# Стратегия №4: TEMA
class TEMAStrategy(Strategy):
    """
    Triple EMA-based strategy
    """

    period = 55

    def init(self) -> None:
        self.close = self.data["Close"]
        ema1 = talib.EMA(self.close, timeperiod=self.period)
        ema2 = talib.EMA(ema1, timeperiod=self.period)
        ema3 = talib.EMA(ema2, timeperiod=self.period)

        # Revealing object
        self.signal = self.I(pd.Series(((3 * ema1) - (3 * ema2) + ema3)).to_numpy)

    def next(self) -> None:
        if self.close[-1] > self.signal[-1]:  # self.signal[-1] == 1:
            if not self.position.is_long:
                self.buy()
        elif self.close[-1] < self.signal[-1]:  # self.signal[-1] == -1:
            if not self.position.is_short:
                self.sell()

In [15]:
# Test a strategy (TEMAStrategy)
bt = Backtest(
    transform_for_backtesting(y_test, X_test),
    TEMAStrategy,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)
stats = bt.run(period=55)
stats[:23]

Start                     2023-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   49.589041
Equity Final [$]                138268.851749
Equity Peak [$]                 144173.973819
Commissions [$]                    466.459124
Return [%]                          38.268852
Buy & Hold Return [%]               30.928759
Return (Ann.) [%]                   38.268852
Volatility (Ann.) [%]               44.493751
CAGR [%]                            38.391992
Sharpe Ratio                         0.860095
Sortino Ratio                        2.006077
Calmar Ratio                         1.654169
Alpha [%]                           28.061759
Beta                                 0.330019
Max. Drawdown [%]                  -23.134787
Avg. Drawdown [%]                    -5.02087
Max. Drawdown Duration      119 days 00:00:00
Avg. Drawdown Duration       15 days 00:00:00
# Trades                          

In [16]:
bt.plot(superimpose=False)

INFO:bokeh.io.state:Session output file 'TEMAStrategy_period-55_.html' already exists, will be overwritten.


c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2445', ...)

In [17]:
# Стратегия №5: Bollinger Bands
class BollingerBandsStrategy(Strategy):
    """
    Bollinger Bands-based strategy
    """

    period = 55

    def init(self) -> None:
        self.close = self.data["Close"]
        self.ma = self.I(talib.SMA, self.close, timeperiod=self.period)
        self.std = self.I(pd.Series(self.close).rolling(window=self.period).std)

        # Upper and lower Bollinger Bands
        self.upperbb_1sd = self.I(pd.Series(self.ma + (1 * self.std)).to_numpy)
        self.upperbb_2sd = self.I(pd.Series(self.ma + (2 * self.std)).to_numpy)
        self.lowerbb_1sd = self.I(pd.Series(self.ma - (1 * self.std)).to_numpy)
        self.lowerbb_2sd = self.I(pd.Series(self.ma - (2 * self.std)).to_numpy)

    def next(self) -> None:
        # Position enter logic
        if self.close[-1] > self.upperbb_1sd[-1]:
            if not self.position.is_long:
                self.buy()
        elif self.close[-1] < self.lowerbb_1sd[-1]:
            if not self.position.is_short:
                self.sell()

        # Position exit logic
        if self.position.is_long:
            if self.close[-1] < self.upperbb_1sd[-1]:
                self.position.close()
            if self.close[-1] > self.lowerbb_1sd[-1]:
                self.position.close()

In [18]:
# Test a strategy (BollingerBandsStrategy)
bt = Backtest(
    transform_for_backtesting(y_test, X_test),
    BollingerBandsStrategy,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)
stats = bt.run(period=55)
stats[:23]

Start                     2023-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   84.383562
Equity Final [$]                 61106.030727
Equity Peak [$]                 103779.620667
Commissions [$]                  43831.910436
Return [%]                         -38.893969
Buy & Hold Return [%]               41.849618
Return (Ann.) [%]                  -38.893969
Volatility (Ann.) [%]               19.562627
CAGR [%]                           -38.976601
Sharpe Ratio                        -1.988177
Sortino Ratio                       -1.808816
Calmar Ratio                        -0.790415
Alpha [%]                          -57.928986
Beta                                 0.454843
Max. Drawdown [%]                  -49.206995
Avg. Drawdown [%]                  -18.897683
Max. Drawdown Duration      290 days 00:00:00
Avg. Drawdown Duration      103 days 00:00:00
# Trades                          

In [19]:
bt.plot(superimpose=False)

c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2861', ...)

# 3. Подбор гиперпараметров моделей с использованием обучающей и тестовой выборок

Обучающая выборка здесь вновь не релевантна, поскольку "модель" в данном контексте имеет заранее определённую логику, а не создаёт внутреннее представление на основе данных из обучающей выборки.
Соответственно, gridsearch и прочие optuna не применимы, необходима самописная реализация.
Реализация - ~то что было продемонстрировано на 10 лекции, но адаптированное под данные в данном проекте

In [1]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os

os.chdir(os.path.dirname(os.getcwd()))

import main
import logging

import itertools
import pandas as pd
from backtesting import Strategy, Backtest

from src.core import utils
from src.strategy.sma_cross_strategy import SmaCross
from src.strategy.macd_strategy import MACDStrategy
from src.strategy.stoch_strategy import STOCHStrategy
from src.strategy.tema_strategy import TEMAStrategy
from src.strategy.bb_strategy import BollingerBandsStrategy

# initialize
logger = logging.getLogger()
# initialize config dict
config = main.main_launch()
# SUBSTITUTE FOR TESTING

c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Params for data download
TICKER = "ETH-USD"
START_DT = "2010-01-01"  # get as long as possible
END_DT = config.END_DT  # defaults to today
INTERVAL = "1d"

# params for split function
train_start = None
train_end = "2023-01-01"
test_end = "2024-01-01"
valid_end = "2025-01-01"  # config.END_DT

In [3]:
# Возьмём данные - с разбивкой на train/test/valid
ticker_data = utils.get_history(
    tickers=TICKER,
    start=START_DT,
    end=END_DT,
    interval=INTERVAL,
    update_cache=False,
)
X_train, y_train, X_test, y_test, X_val, y_val = utils.train_test_valid_split(
    ticker_data, train_start, train_end, test_end, valid_end, drop_leaky=False
)

[INFO   ] 2025-03-24@16:07:57: Getting history from local cache DB...
[INFO   ] 2025-03-24@16:07:58: Got history of shape (2690, 7), 0 NaNs
[INFO   ] 2025-03-24@16:07:58: Splitting ticker data to train/test/validation parts


In [4]:
# 1. Нам нужна функция, которая будет производить 1 итерацию бэктеста, после ее зациклим
def backtest_strategy(strategy_class, y_test:pd.Series, X_test:pd.DataFrame, strategy_params:dict):
    """
    Perform a backtest of strategy_class with params on test dataset
    """
    # Initialize Backtest object
    bt = Backtest(
        utils.transform_for_backtesting(y_test=y_test, X_test=X_test),
        strategy_class,
        cash=100000,
        commission=0.002,
        exclusive_orders=True
    )
    # Run, using selected Strategy parameters
    stats = bt.run(**strategy_params)
    return stats

In [5]:
# 2. Функция для нахождения лучшего набора параметров для ОДНОГО типа стратегии
def get_best_strategy_params(strategy_class, y_test:pd.Series, X_test:pd.DataFrame, strategy_params_options:dict, kpi:str="Return [%]"):
    """
    Iterate over combinations of strategy_params_options for strategy_class
    Perform backtesting experiment for each of them, and return best (on "Return [%]") params and performance
    """
    # Variables to hold best params
    best_params = None
    best_performance = -float('inf')

    for param_option in itertools.product(*strategy_params_options.values()):
        # Generate strategy params from options
        strategy_params = dict(zip(strategy_params_options.keys(), param_option))
        # logger.info(f"Testing with params: {strategy_params}")

        # Perform backtest
        stats = backtest_strategy(
            strategy_class,
            y_test=y_test,
            X_test=X_test,
            strategy_params=strategy_params,
        )

        # Get key performance indicator among all metrics
        performance = stats[kpi]

        # Check if it's better than others
        if performance > best_performance:
            best_performance = performance
            best_params = strategy_params

    logger.info(f"Best Performance: {best_performance}")
    logger.info(f"Best Parameters: {best_params}")

    return best_params, best_performance


In [6]:
# 3. Функция для нахождения лучшего набора параметров и типа стратегии, включая выбор среди нескольких из них
def get_best_strategy(full_strategy_test_list:dict, y_test:pd.Series, X_test:pd.DataFrame):
    # A dictionary to return all bests for strategies
    full_test_summary = {}

    # Variables to hold best params
    best_strategy_class = None
    best_params = None
    best_performance = -float('inf')

    for e in full_strategy_test_list:
        logger.info("= = = = = = = = = = = = = = = = = = = = = = = =")
        logger.info(f"Searching best params for {e['strategy_type']}...")
        class_params, class_performance = get_best_strategy_params(
            e['strategy_class'],
            y_test=y_test,
            X_test=X_test,
            strategy_params_options=e['strategy_params_options'],
        )

        full_test_summary[e['strategy_type']] = {'strategy_class': e['strategy_class'], 'params': class_params, 'performance': class_performance}

        # Check if it's better than others
        if class_performance > best_performance:
            best_performance = class_performance
            best_params = class_params
            best_strategy_class = e['strategy_class']

    logger.info("= = = = = = = = = = = = = = = = = = = = = = = =")
    logger.info(f"Best Strategy Class: {str(best_strategy_class)}")
    logger.info(f"Best Performance: {best_performance}")
    logger.info(f"Best Parameters: {best_params}")
    logger.info("= = = = = = = = = = = = = = = = = = = = = = = =")

    return best_strategy_class, best_params, best_performance, full_test_summary

In [7]:
# Список моделей и их гиперпараметров
full_strategy_test_list = [
    {
        "strategy_type": "SmaCross",
        "strategy_class": SmaCross,
        "strategy_params_options": {
            "ma_fast_periods": [2, 3, 5, 7, 10],
            "ma_slow_periods": [5, 7, 10, 14, 20, 30],
        },
    },
    {
        "strategy_type": "MACDStrategy",
        "strategy_class": MACDStrategy,
        "strategy_params_options": {
            "fastperiod": [3, 5, 7, 14, 20],
            "slowperiod": [14, 20, 26, 30, 40],
            "signalperiod": [7, 9, 11, 14],
        },
    },
    {
        "strategy_type": "STOCHStrategy",
        "strategy_class": STOCHStrategy,
        "strategy_params_options": {
            "fastk_period": [7, 10, 14, 20],
            "slowk_period": [5, 7, 10],
            "slowk_matype": [0],
            "slowd_period": [3, 7, 10],
            "slowd_matype": [0],
        },
    },
    {
        "strategy_type": "TEMAStrategy",
        "strategy_class": TEMAStrategy,
        "strategy_params_options": {
            "period": [14, 21, 28, 40, 55, 70, 90],
        },
    },
    {
        "strategy_type": "BollingerBandsStrategy",
        "strategy_class": BollingerBandsStrategy,
        "strategy_params_options": {
            "period": [14, 21, 28, 40, 55, 70, 90],
        },
    },
]

In [8]:
# Проведём тестирование
best_strategy_class, best_params, best_performance, full_test_summary = (
    get_best_strategy(full_strategy_test_list, y_test=y_test, X_test=X_test)
)

[INFO   ] 2025-03-24@16:10:44: = = = = = = = = = = = = = = = = = = = = = = = =
[INFO   ] 2025-03-24@16:10:44: Searching best params for SmaCross...
[INFO   ] 2025-03-24@16:10:45: Best Performance: 35.37140273535159
[INFO   ] 2025-03-24@16:10:45: Best Parameters: {'ma_fast_periods': 3, 'ma_slow_periods': 30}
[INFO   ] 2025-03-24@16:10:45: = = = = = = = = = = = = = = = = = = = = = = = =
[INFO   ] 2025-03-24@16:10:45: Searching best params for MACDStrategy...
[INFO   ] 2025-03-24@16:10:50: Best Performance: 0.0
[INFO   ] 2025-03-24@16:10:50: Best Parameters: {'fastperiod': 14, 'slowperiod': 14, 'signalperiod': 7}
[INFO   ] 2025-03-24@16:10:50: = = = = = = = = = = = = = = = = = = = = = = = =
[INFO   ] 2025-03-24@16:10:50: Searching best params for STOCHStrategy...
[INFO   ] 2025-03-24@16:10:52: Best Performance: 69.95620107568364
[INFO   ] 2025-03-24@16:10:52: Best Parameters: {'fastk_period': 14, 'slowk_period': 5, 'slowk_matype': 0, 'slowd_period': 3, 'slowd_matype': 0}
[INFO   ] 2025-03

In [9]:
full_test_summary

{'SmaCross': {'strategy_class': src.strategy.sma_cross_strategy.SmaCross,
  'params': {'ma_fast_periods': 3, 'ma_slow_periods': 30},
  'performance': 35.37140273535159},
 'MACDStrategy': {'strategy_class': src.strategy.macd_strategy.MACDStrategy,
  'params': {'fastperiod': 14, 'slowperiod': 14, 'signalperiod': 7},
  'performance': 0.0},
 'STOCHStrategy': {'strategy_class': src.strategy.stoch_strategy.STOCHStrategy,
  'params': {'fastk_period': 14,
   'slowk_period': 5,
   'slowk_matype': 0,
   'slowd_period': 3,
   'slowd_matype': 0},
  'performance': 69.95620107568364},
 'TEMAStrategy': {'strategy_class': src.strategy.tema_strategy.TEMAStrategy,
  'params': {'period': 14},
  'performance': 79.25398813867184},
 'BollingerBandsStrategy': {'strategy_class': src.strategy.bb_strategy.BollingerBandsStrategy,
  'params': {'period': 40},
  'performance': -21.74908141357421}}

In [12]:
# Replicate best strategy
bt = Backtest(
    utils.transform_for_backtesting(y_test, X_test),
    best_strategy_class,
    cash=100000,
    commission=0.002,
    exclusive_orders=True,
)
stats = bt.run(**best_params)
stats[:23]

Start                     2023-01-01 00:00:00
End                       2023-12-31 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   87.945205
Equity Final [$]                179253.988139
Equity Peak [$]                 179253.988139
Commissions [$]                     1762.7239
Return [%]                          79.253988
Buy & Hold Return [%]               47.530703
Return (Ann.) [%]                   79.253988
Volatility (Ann.) [%]               79.805868
CAGR [%]                            79.541633
Sharpe Ratio                         0.993085
Sortino Ratio                        3.064639
Calmar Ratio                         2.922893
Alpha [%]                           42.329143
Beta                                 0.776863
Max. Drawdown [%]                  -27.114909
Avg. Drawdown [%]                   -4.724186
Max. Drawdown Duration      207 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                          

In [13]:
bt.plot(superimpose=False)

c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\ProgramData\miniconda3\envs\hw_env\lib\site-packages\backtesting\_plotting.py:700: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1307', ...)